In [7]:
# ---------------------- IMPORTS & SETUP ----------------------
from google import genai
from google.genai import types
import wave
from google.colab import userdata
from PIL import Image
from io import BytesIO
import base64
import numpy as np
from IPython.display import display, Image as IPImage, Audio


# Authenticate Gemini client
client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))

# ---------------------- AUDIO HELPERS ----------------------
def wave_file(filename, pcm, channels=1, rate=24000, sample_width=2):
    with wave.open(filename, "wb") as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(sample_width)
        wf.setframerate(rate)
        wf.writeframes(pcm)

def generate_tts(text, voice_name="Kore", file_name="tts_output.wav"):
    response = client.models.generate_content(
        model="gemini-2.5-flash-preview-tts",
        contents=text,
        config=types.GenerateContentConfig(
            response_modalities=["AUDIO"],
            speech_config=types.SpeechConfig(
                voice_config=types.VoiceConfig(
                    prebuilt_voice_config=types.PrebuiltVoiceConfig(voice_name=voice_name)
                )
            ),
        ),
    )
    data = response.candidates[0].content.parts[0].inline_data.data
    wave_file(file_name, data)
    print(f"🔉 Audio saved to '{file_name}'")

# ---------------------- IMAGE GENERATION ----------------------
def generate_flashcard_image(word, lang):
    """
    Generates an image for a flashcard based on a word in the target language.
    """
    #First, translate the English word to the target language to ensure the image context is correct.
    translation_prompt = f"Translate the English word '{word}' into {lang}. Provide only the single translated word or short phrase."
    translated_word = ask_gemini(translation_prompt)
    print(f"   (Generating image for '{translated_word}'...)")
    # ------------------------------------

    #Uses the translated word for more accurate image generation.
    prompt = (
        f"Create a simple, colorful illustration representing '{translated_word}' (the {lang} word for '{word}'). "
        f"The image should be suitable for a language flashcard. Use a clean, plain background. "
        "The main subject should be centered. **Do not include any text in the image.**"
    )
    try:
        response = client.models.generate_content(
            model="gemini-2.0-flash-preview-image-generation",
            contents=prompt,
            config=types.GenerateContentConfig(
                response_modalities=['TEXT' ,'IMAGE']
            ),
        )
        for part in response.candidates[0].content.parts:
            if part.inline_data is not None:
                return part.inline_data.data
    except Exception as e:
        print(f"❌ Error generating image: {e}")
    return None

# ---------------------- TEXT RESPONSE ----------------------
def ask_gemini(prompt):
    response = client.models.generate_content(
        model="gemini-1.5-flash",
        contents=prompt
    )
    return response.candidates[0].content.parts[0].text.strip()

# ---------------------- MAIN MENU ----------------------
def main_menu():
    print("👋 Welcome to your AI Language Coach!")
    target_lang = input("🌐 What language would you like to learn? (e.g., Spanish, Japanese): ").capitalize()

    while True:
        print(f"\n📚 Learning: {target_lang}")
        print("====== MAIN MENU ======")
        print("1. Practice Conversation")
        print("2. Create Visual Flashcards")
        print("3. Hear Pronunciation")
        print("4. Play Vocabulary Game")
        print("5. Change Language")
        print("6. Exit")
        choice = input("🔸 Select (1–6): ")

        if choice == "1":
            practice_conversation(target_lang)
        elif choice == "2":
            create_flashcards(target_lang)

        elif choice == "3":
            hear_pronunciation(target_lang)
        elif choice == "4":
            play_vocabulary_game(target_lang)
        elif choice == "5":
            target_lang = input("🔄 Enter new language: ").capitalize()
        elif choice == "6":
            print("👋 Goodbye! Keep practicing!")
            break
        else:
            print("❌ Invalid option. Try again!")

# ---------------------- FEATURES ----------------------
def practice_conversation(lang):
    print(f"\n💬 Practice mode in {lang}. Type 'exit' to go back to the menu.")
    while True:
        user_input = input("🗨️ You: ")
        if user_input.lower() == "exit":
            break

        # Step 1: AI reply in target language
        reply_prompt = (
            f"You are a friendly {lang} teacher helping a beginner student. "
            f"Respond to this input with a short, simple sentence in {lang}: '{user_input}'"
        )
        reply = ask_gemini(reply_prompt)

        # Step 2: Translate the AI reply
        translation_prompt = f"Translate this {lang} sentence to English: {reply}"
        reply_translation = ask_gemini(translation_prompt)

        # Step 3: Explain 1–2 key vocabulary words
        vocab_prompt = (
            f"Explain the 1–2 most important vocabulary words from this {lang} sentence: '{reply}'. "
            f"Give the word, its English meaning, and a short usage example in {lang}."
        )
        vocab_explanation = ask_gemini(vocab_prompt)

        # Step 4: Ask a follow-up question in target language
        follow_up_prompt = (
            f"As a language teacher, ask a beginner-friendly follow-up question in {lang} "
            f"based on this sentence: '{user_input}'"
        )
        follow_up = ask_gemini(follow_up_prompt)

        # Step 5: Translate the follow-up question
        follow_up_translation_prompt = f"Translate this {lang} question to English: {follow_up}"
        follow_up_translation = ask_gemini(follow_up_translation_prompt)

        # Display output clearly
        print(f"\n🤖 {lang} Coach says: {reply}")
        print(f"   💬 (Translation): {reply_translation}")
        print(f"   📘 Key vocabulary:\n{vocab_explanation}")
        print(f"\n❓ Follow-up Question: {follow_up}")
        print(f"   💬 (Translation): {follow_up_translation}")

def create_flashcards(lang):
    print(f"\n🧠 Let's create flashcards in {lang}!")
    words = input("Enter words (comma-separated) you want to learn: ")
    word_list = [w.strip() for w in words.split(",")]

    print("\n📇 Flashcards:")
    for i, word in enumerate(word_list):
        # Get translation and example
        prompt = f"Translate the word '{word}' into {lang} and use it in a simple sentence."
        flashcard = ask_gemini(prompt)
        print(f"\n🔹 {word} ➡️ {flashcard}")

        # Generate and display image
        try:
            image_data = generate_flashcard_image(word, lang)
            if image_data:
                display(IPImage(data=image_data))
                # Save image to file
                with open(f"flashcard_{i}.png", "wb") as f:
                    f.write(image_data)
            else:
                print("❌ Image generation failed")
        except Exception as e:
            print(f"❌ Error generating image: {e}")


def hear_pronunciation(lang):
    print(f"\n🔊 Pronunciation in {lang}")
    word = input("Type a word or phrase: ")
    generate_tts(f"This is how you say '{word}' in {lang}. {word}", voice_name="Kore")
    audio = Audio(filename="tts_output.wav")
    display(audio)

def play_vocabulary_game(lang):
    print(f"\n🎲 Let's play a vocabulary game in {lang}!")
    topic = input("Choose a topic (e.g., food, travel, greetings): ")
    prompt = f"Create a simple vocabulary matching game with 5 {lang} words and their English meanings about {topic}. Format it like a quiz."
    game = ask_gemini(prompt)
    print("🧩 Game:\n")
    print(game)

# ---------------------- RUN ----------------------
main_menu()

👋 Welcome to your AI Language Coach!
🌐 What language would you like to learn? (e.g., Spanish, Japanese): spanish

📚 Learning: Spanish
====== MAIN MENU ======
1. Practice Conversation
2. Create Visual Flashcards
3. Hear Pronunciation
4. Play Vocabulary Game
5. Change Language
6. Exit
🔸 Select (1–6): 5
🔄 Enter new language: russian

📚 Learning: Russian
====== MAIN MENU ======
1. Practice Conversation
2. Create Visual Flashcards
3. Hear Pronunciation
4. Play Vocabulary Game
5. Change Language
6. Exit
🔸 Select (1–6): 3

🔊 Pronunciation in Russian
Type a word or phrase: Привет мамочка
🔉 Audio saved to 'tts_output.wav'



📚 Learning: Russian
====== MAIN MENU ======
1. Practice Conversation
2. Create Visual Flashcards
3. Hear Pronunciation
4. Play Vocabulary Game
5. Change Language
6. Exit


KeyboardInterrupt: Interrupted by user